In [17]:
import cv2 as cv
import numpy as np
import os

In [18]:
def show_image(title,image):
    image=cv.resize(image,(0,0),fx=0.2,fy=0.2)
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [19]:
def extrage_careu(img,clas=None):

    low = (85, 0, 0)
    high = (108, 255, 255)
    img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    mask= cv.inRange(img_hsv, low, high)
    #show_image('img_initial', img)
    res = cv.bitwise_and(img, img, mask=mask) 
    #show_image("hsv",res)


    image_gray = cv.cvtColor(res,cv.COLOR_BGR2GRAY)
    #show_image("gray",image_gray)

    image_m_blur = cv.medianBlur(image_gray,27)
    image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 9) 
    image_sharpened = cv.addWeighted(image_m_blur, 2, image_g_blur, -0, 0)
    #show_image('image_sharpened',image_sharpened)

    _, thresh = cv.threshold(image_sharpened, 10, 255, cv.THRESH_BINARY)
    
    #show_image('image_thresholded',thresh)

    kernel = np.ones((5, 5), np.uint8)
    thresh = cv.erode(thresh, kernel)
    #show_image('image_thresholded',thresh)
    #cv.imwrite("thresh_photo.jpg",thresh)

    edges =  cv.Canny(thresh ,100,175)
    #show_image('edges',edges)
    
    contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)


    max_area=0
    for i in range(len(contours)):
        if(len(contours[i]) >3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]

            current_area = cv.contourArea(np.array([[possible_top_left], [possible_top_right], [possible_bottom_right], [possible_bottom_left]]))

         
            width = np.linalg.norm(possible_top_right - possible_top_left)
            height = np.linalg.norm(possible_bottom_left - possible_top_left)
            aspect_ratio = width / height if height != 0 else 0

         
            if current_area > max_area and abs(1 - aspect_ratio) < 0.1:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left

    width = 2700
    height = 2700

    image_copy = cv.cvtColor(image_gray.copy(),cv.COLOR_GRAY2BGR)
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    #show_image("detected corners",image_copy)
    
    destination=np.array([[0,0],[width,0],[width,height],[0,height]],dtype="float32")
    
    if clas:
        square_size_increase = 180
        width +=2*square_size_increase
        height +=2*square_size_increase

        destination = np.array(
            [
                [0 + square_size_increase, 0 + square_size_increase],
                [width - square_size_increase, 0 + square_size_increase],
                [width - square_size_increase, height - square_size_increase],
                [0 + square_size_increase, height - square_size_increase],
            ],
            dtype="float32",
        )


    puzzle=np.array([top_left,top_right,bottom_right,bottom_left],dtype="float32")

    M=cv.getPerspectiveTransform(puzzle,destination)
    result=cv.warpPerspective(image_gray,M,[width,height])

    result=cv.cvtColor(result,cv.COLOR_GRAY2BGR)
    #cv.imwrite("result.jpg",result)
    
    cv.waitKey(0)
    cv.destroyAllWindows()
    
    return result





In [20]:
lines_horizontal=[]
for i in range(0,2701,180):
    l=[]
    l.append((0,i))
    l.append((2699,i))
    lines_horizontal.append(l)

In [21]:
lines_vertical=[]
for i in range(0,2701,180):
    l=[]
    l.append((i,0))
    l.append((i,2699))
    lines_vertical.append(l)

In [22]:
def vizualizare_configuratie(result,matrix,lines_horizontal,lines_vertical):
    for i in range(len(lines_horizontal) - 1):
        for j in range(len(lines_vertical) - 1):
            y_min = lines_vertical[j][0][0]+180
            y_max = lines_vertical[j + 1][1][0]+180
            x_min = lines_horizontal[i][0][1]+180
            x_max = lines_horizontal[i + 1][1][1]+180
            if matrix[i][j] != -1: 
                cv.rectangle(result, (y_min, x_min), (y_max, x_max), color=(255, 0, 0), thickness=5)

In [23]:
def determina_configuratie_careu(img,matrix,thresh,lines_horizontal,lines_vertical):
    i1=0
    i2=0
    j1=0
    j2=0
    c=0
    numbers=[0,1,2,3,4,5,6]
    for i in range(len(lines_horizontal)-1):
        for j in range(len(lines_vertical)-1):
            y_min = lines_vertical[j][0][0]+180
            y_max = lines_vertical[j + 1][1][0]+180
            x_min = lines_horizontal[i][0][1]+180
            x_max = lines_horizontal[i + 1][1][1]+180
            y_min-=10
            y_max+=10
            x_min-=10
            x_max+=10
            patch = thresh[x_min:x_max, y_min:y_max].copy()
            patch_orig=img[x_min:x_max, y_min:y_max].copy()
            patch_orig= cv.cvtColor(patch_orig,cv.COLOR_BGR2GRAY)
            Medie_patch=np.mean(patch)
            if Medie_patch<140:
                if matrix[i][j]==-1:
                    matrix[i][j]=clasifica_cifra(patch_orig,1,None)
                    if  matrix[i][j]==-1:
                        continue
                    c+=1
                    if c==1:
                        i1=i
                        j1=j
                    if c==2:
                        i2=i
                        j2=j
                        
                        
    y_min = lines_vertical[j1][0][0]+180
    y_max = lines_vertical[j2+1][1][0]+180
    x_min = lines_horizontal[i1][0][1]+180
    x_max = lines_horizontal[i2+1][1][1]+180
    y_min-=30
    y_max+=30
    x_min-=30
    x_max+=30
    patch_orig=img[x_min:x_max, y_min:y_max].copy()
    patch_orig= cv.cvtColor(patch_orig,cv.COLOR_BGR2GRAY)
    if j1<j2:
        val=(clasifica_cifra(patch_orig,2,"orizontal"))
        matrix[i1][j1]=val[0]
        matrix[i2][j2]=val[1]
    if i1<i2:
        val=(clasifica_cifra(patch_orig,2,"vertical"))
        matrix[i1][j1]=val[1]
        matrix[i2][j2]=val[0]
        
    return matrix,(matrix[i1][j1],i1,j1),(matrix[i2][j2],i2,j2)

In [24]:
def board():
    matrix = np.full((15,15), 0)
    matrix[0][0]=5
    matrix[14][14]=5
    matrix[0][14]=5
    matrix[14][0]=5
    
    matrix[0][3]=4
    matrix[0][11]=4
    matrix[1][5]=4
    matrix[1][9]=4
    matrix[3][0]=4
    matrix[3][14]=4
    matrix[5][1]=4
    matrix[5][13]=4
    matrix[9][1]=4
    matrix[9][13]=4
    matrix[11][0]=4
    matrix[11][14]=4
    matrix[13][5]=4
    matrix[13][9]=4
    matrix[14][3]=4
    matrix[14][11]=4
    
    matrix[0][7]=3
    matrix[1][2]=3
    matrix[1][12]=3
    matrix[2][1]=3
    matrix[2][13]=3
    matrix[3][3]=3
    matrix[3][11]=3
    matrix[7][0]=3
    matrix[7][14]=3
    matrix[14][7]=3
    matrix[13][2]=3
    matrix[13][12]=3
    matrix[12][1]=3
    matrix[12][13]=3
    matrix[11][3]=3
    matrix[11][11]=3
    
    matrix[2][4]=2
    matrix[2][10]=2
    matrix[3][5]=2
    matrix[3][9]=2
    matrix[4][2]=2
    matrix[4][12]=2
    matrix[5][3]=2
    matrix[5][11]=2
    matrix[12][4]=2
    matrix[12][10]=2
    matrix[11][5]=2
    matrix[11][9]=2
    matrix[10][2]=2
    matrix[10][12]=2
    matrix[9][3]=2
    matrix[9][11]=2
    
    matrix[4][4]=1
    matrix[4][6]=1
    matrix[4][8]=1
    matrix[4][10]=1
    matrix[5][5]=1
    matrix[5][9]=1
    matrix[6][4]=1
    matrix[6][10]=1
    matrix[10][4]=1
    matrix[10][6]=1
    matrix[10][8]=1
    matrix[10][10]=1
    matrix[9][5]=1
    matrix[9][9]=1
    matrix[8][4]=1
    matrix[8][10]=1
    
    poz=[-1,1,2,3,4,5,6,0,2,5,3,4,6,2,2,0,3,5,4,1,6,2,4,5,5,0,6,3,4,2,0,1,5,1,3,4,4,4,5,0,6,3,
             5,4,1,3,2,0,0,1,1,2,3,6,3,5,2,1,0,6,6,5,2,1,2,5,0,3,3,5,0,6,1,4,0,6,3,
             5,1,4,2,6,2,3,1,6,5,6,2,0,4,0,1,6,4,4,1,6,6,3,0]
    return poz,matrix
    
    
    

In [25]:
def clasifica_cifra(patch,x,tip):
    if x==1:
        
        maxi=-np.inf
        poz=-1
        rotation=[0,90,180,270]
        letters=["do","ub","le"]
        for i in rotation:
            for j in range(0,7):
                img_template=cv.imread('template/single/'+str(j)+'_'+str(i)+'.jpg')
                img_template= cv.cvtColor(img_template,cv.COLOR_BGR2GRAY)
                corr = cv.matchTemplate(patch,img_template,  cv.TM_CCOEFF_NORMED)
                corr=np.max(corr)
                if corr>maxi:
                    maxi=corr
                    poz=j
                    
        for c in letters:            
            img_template=cv.imread('template/single/'+c+'.jpg')
            img_template= cv.cvtColor(img_template,cv.COLOR_BGR2GRAY)
            corr = cv.matchTemplate(patch,img_template,  cv.TM_CCOEFF_NORMED)
            corr=np.max(corr)
            if corr>maxi:
                maxi=corr
                poz=-1

        return poz

    if x==2:
        maxi=-np.inf
        poz=(-1,-1)
        for i in range(0,7):
            for j in range(0,7):
                img_template=cv.imread('template/'+tip+'/'+str(j)+'_'+str(i)+'.jpg')
                img_template= cv.cvtColor(img_template,cv.COLOR_BGR2GRAY)
                corr = cv.matchTemplate(patch,img_template,  cv.TM_CCOEFF_NORMED)
                corr=np.max(corr)
                if corr>maxi:
                    maxi=corr
                    poz=(j,i)
        return poz
    


In [26]:
def run(input_file):
    letters="ABCDEFGHIJKLMNO"
    for i in range(1,6):
        players=[]
        with open(input_file+str(i)+"_mutari.txt", 'r') as file:
            for line in file:
                players.append(line)
        player1=0
        player2=0
        index=0
        poz, tabla=board()
        files=os.listdir(input_file)
        matrix = np.full((15,15), -1)
        for file in files:
            if file[-3:]=='jpg' and file[0]==str(i):
                p=players[index].split()[1]
                img = cv.imread(input_file+file)
                result=extrage_careu(img,clas=True)
                _, thresh = cv.threshold(result, 190, 255, cv.THRESH_BINARY_INV)
                #cv.imwrite("tabla_an.jpg",thresh)
                matrix,d1,d2=determina_configuratie_careu(result,matrix,thresh,lines_horizontal,lines_vertical)
                
                
                init1=player1
                init2=player2
                
                if poz[player1]== d1[0] or poz[player1]==d2[0]:
                    player1+=3
                if poz[player2]== d2[0] or poz[player2]==d1[0]:
                    player2+=3
                
                
                rez=0
                init=0
                if p=="player1":
                    player1+=(tabla[d1[1]][d1[2]]+tabla[d2[1]][d2[2]])*2 if d1[0]==d2[0] else (tabla[d1[1]][d1[2]]+tabla[d2[1]][d2[2]])
                    rez=player1
                    init=init1
                else:
                    player2+=(tabla[d1[1]][d1[2]]+tabla[d2[1]][d2[2]])*2 if d1[0]==d2[0] else (tabla[d1[1]][d1[2]]+tabla[d2[1]][d2[2]])
                    rez=player2
                    init=init2
                
                poz1=str(d1[1]+1)+letters[d1[2]]
                poz2=str(d2[1]+1)+letters[d2[2]]
                
                with open("rezultate/"+file[0:5]+"txt", "w") as f:
                    f.write(str(poz1) + ' ' + str(d1[0]) + '\n')
                    f.write(str(poz2) + ' ' + str(d2[0]) + '\n')
                    f.write(str(rez - init))

                index+=1
                #vizualizare_configuratie(result,matrix,lines_horizontal,lines_vertical)

run()
    